In [ ]:
! pip install langchain
! pip install openai
! pip install PyPDF2
! pip install faiss-cpu
! pip install tiktoken

In [3]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS

In [4]:
# location of the pdf file/files.
reader = PdfReader('/Users/adsonalves/Desktop/Cloud/faturas/lei143002022.pdf')

In [5]:
# read data from the file and put them into a variable called raw_text
raw_text = ''
for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

In [6]:
raw_text

" Ministério de Minas e Energia  \nConsultoria Jurídica  \n \nLEI No 14.300, DE 6 DE JANEI RO DE 20 22. \n \nInstitui o marco legal da microgeração e minigeração  \ndistribuída, o Sistema de Compensação de Energia Elétrica \n(SCEE) e o Programa de Energia Renovável Social (PERS); \naltera as Leis nºs 10.848, de 15 de março de 2004, e 9.427, \nde 26 de dezembro de 1996; e dá outras providências.  \n \nO PRESIDENTE DA  REPÚBLICA  Faço saber que o Congresso Nacional decreta e eu sanciono a \nseguinte Lei:  \nCAPÍTULO I  \nDISPOSIÇÕES PRELIMINARES  \nArt. 1º   Para fins e efeitos desta Lei, são adotadas as seguintes definições:  \nI - autoconsumo local: modalidade de microgeração ou minigeração  distribuída eletricamente \njunto à carga, participante do Sistema de Compensação de Energia Elétrica (SCEE), no qual o excedente \nde energia elétrica gerado por unidade consumidora de titularidade de um consumidor -gerador, pessoa \nfísica ou jurídica, é compe nsado ou creditado pela mesma unidad

In [7]:
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [8]:
texts[0]

'Ministério de Minas e Energia  \nConsultoria Jurídica  \n \nLEI No 14.300, DE 6 DE JANEI RO DE 20 22. \n \nInstitui o marco legal da microgeração e minigeração  \ndistribuída, o Sistema de Compensação de Energia Elétrica \n(SCEE) e o Programa de Energia Renovável Social (PERS); \naltera as Leis nºs 10.848, de 15 de março de 2004, e 9.427, \nde 26 de dezembro de 1996; e dá outras providências.  \n \nO PRESIDENTE DA  REPÚBLICA  Faço saber que o Congresso Nacional decreta e eu sanciono a \nseguinte Lei:  \nCAPÍTULO I  \nDISPOSIÇÕES PRELIMINARES  \nArt. 1º   Para fins e efeitos desta Lei, são adotadas as seguintes definições:  \nI - autoconsumo local: modalidade de microgeração ou minigeração  distribuída eletricamente \njunto à carga, participante do Sistema de Compensação de Energia Elétrica (SCEE), no qual o excedente \nde energia elétrica gerado por unidade consumidora de titularidade de um consumidor -gerador, pessoa \nfísica ou jurídica, é compe nsado ou creditado pela mesma unidade

In [19]:
embeddings = OpenAIEmbeddings()
docsearch = FAISS.from_texts(texts, embeddings)

In [20]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
chain = load_qa_chain(OpenAI(), chain_type="stuff")

In [14]:
query = "O que é energia injetada e compensada?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' Energia injetada é a energia elétrica gerada por unidade consumidora de titularidade de um consumidor-gerador, pessoa física ou jurídica, que é compensada ou creditada pela mesma unidade consumidora.'

In [18]:
query = "O que é geração compartilhada ?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' Geração compartilhada é uma modalidade caracterizada pela reunião de consumidores, por meio de consórcio, cooperativa, condomínio civil voluntário ou edilício ou qualquer outra forma de associação civil, instituída para esse fim, composta por pessoas físicas ou jurídicas que possuam unidade consumidora com microgeração ou minigeração distribuída.'

In [16]:
query = "Qual valor de tarifa ?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' O valor de tarifa deve ser considerado de acordo com as regras aplicáveis às unidades consumidoras do mesmo nível de tensão até a revisão tarifária da distribuidora subsequente à publicação desta Lei. No caso de unidades consumidoras com minigeração distribuída pertencentes e faturadas no Grupo A, a tarifa corresponde à forma de uso do sistema de distribuição realizada pela unidade com microgeração ou minigeração distribuída, se para injetar ou consumir energia.'

In [22]:
query = "Quanto tempo para ter a energia compensada?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-embedding-ada-002 in organization org-P7NJ2wjV5vUpciXAMPef2mFd on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..


' 30 dias.'